In [1]:
#Importamos las librerias
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import time
from numpy import linalg as LA
import random
from time import time
%matplotlib qt

In [2]:
#Definimos la funcion a minimizar
def f(x):
    fx = 100*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 + 90*(np.sqrt(x[0]**2+(x[1]+1)**2)-1)**2 -(20*x[0]+40*x[1])
    return fx

In [3]:
#Definir funcion para calcular la matriz del gradiente
def gradient(x,delta):
    grad=np.zeros(2)
    grad[0]=(f([x[0]+delta,x[1]])- f([x[0]-delta,x[1]]))/(2*delta)
    grad[1]=(f([x[0],x[1]+delta])- f([x[0],x[1]-delta]))/(2*delta)
    return grad

In [4]:
#Definir funcion para calcular la matriz Hessiana
def Hessian(x,delta):
    H=np.zeros([2,2])
    H[0,0]= (f([x[0]+delta,x[1]]) - 2*f([x[0],x[1]]) +  f([x[0]-delta,x[1]]))/(delta**2)
    H[1,1]= (f([x[0],x[1]+delta]) - 2*f([x[0],x[1]]) +  f([x[0],x[1]-delta]))/(delta**2)
    H[0,1]= (f([x[0]+delta,x[1]+delta]) - f([x[0]+delta,x[1]-delta]) - f([x[0]-delta,x[1]+delta]) +  f([x[0]-delta,x[1]-delta]))/(4*delta**2)
    H[1,0]= H[0,1]
    return H

In [5]:
#Utilizamos el algoritmo de golden section para encontrar alpha optimo
def golden(x,search,xi,eps):
    a = xi[0];
    b = xi[1];
    tau = 0.381967;
    alpha1 = a*(1-tau) + b*tau;
    alpha2 = a*tau + b*(1-tau);
    falpha1 = f(x+alpha1*search);
    falpha2 = f(x+alpha2*search);
    for i in range(100):
        if falpha1 > falpha2:
            a = alpha1;
            alpha1 = alpha2;
            falpha1 = falpha2;
            alpha2 = tau*a + (1-tau)*b;
            falpha2 = f(x+alpha2*search);
        else:
            b = alpha2;
            alpha2 = alpha1;
            falpha2 = falpha1;
            alpha1 = tau*b + (1-tau)*a;
            falpha1 = f(x+alpha1*search);
        if np.abs(f(x+alpha1*search)- f(x+alpha2*search)) < eps :
            break;
    return alpha1,falpha1

In [6]:
#Graficamos la funcion de prueba
X1 = np.arange(-5, 5, 0.01)
X2 = np.arange(-5, 5, 0.01)
x1, x2 = np.meshgrid(X1, X2)
z = f([x1,x2])
fig = plt.figure()
ax = fig.add_subplot(projection = '3d')
surf = ax.plot_surface(x1, x2, z, cmap=cm.autumn, linewidth=0, antialiased = False, alpha=0.2)
plt.show()

In [7]:
#Variable en comun de los métodos
delta = 1e-3 
ep1 = 1e-3
puntosM = []

In [8]:
def graficaUnaaUna(puntos, nombreMetodo):
    #Generamos valores ramdom para generar un color diferente de grafica
    r = random.random()
    b = random.random()
    g = random.random()
    color = (r, g, b)
    aux = 0
    x = np.linspace(-1, 1, 100)
    y = np.linspace(-1, 1, 100)
    xx, yy = np.meshgrid(x, y)
    z = f([xx, yy])
    plt.contour(x,y,z,20)
    plt.plot(-1,-1, 'ro--',  markersize=6)
    plt.plot(0.5,0, 'go--', markersize=6)
    #Graficamos los trazos (direcciones) que va generando el algoritmo
    for pts in puntos:
        if(aux==len(puntos)-1):
            plt.plot(pts[0],pts[1], c=color, linewidth=2, label=str(nombreMetodo))
        else:
            plt.plot(pts[0],pts[1], c=color, linewidth=2)
        aux = aux + 1
    plt.legend()    
    plt.show

In [10]:
start_time = time() #Conteo para tiempo
iteraciones = 0 #Conteo de numero de iteraciones
I = np.identity(2) #Se establece la matriz identidad 2x2
labda = 10000 #Se define un labda inicial
x = [-1,1] #Punto de partida
x_prev = x #Definimos una variable de punto anterior
fx_prev=f(x) #Se evalua la funcion con el punto inicial
print('Valor de la función inicial = %f ' % fx_prev)
for i in range(1000): #Proceso iterativo del algoritmo
    xAntLM = x #Almacenamos el punto anterior
    G = gradient(x,delta) #Se calcula el vector gradiente
    H = Hessian(x,delta) #Se calcula la matriz Hessiana
    S = -np.matmul(np.linalg.inv((H+labda*I)),G) #Se realiza la multiplicacion del resultado la inversa Hessiana summada la matriz Identidad multiplicada por labda, el resultado es multiplicado por el gradiente de la funcion
    x=x+S #Se actuliza el nuevo punto optimo
    fx=f(x) #Se evalua la funcion con el nuevo punto optimo
    if(fx<fx_prev): #Si fx es menor al previo la nueva labda sera multiplicada por 0.5 y se tendra un nuevo tamaño de paso
        labda=.5*labda
    else: #Caso contrario labda sera multiplicado por 2
        labda=2*labda
    #Almacenamos los puntos de direccion para trazar la grafoica
    puntosM.append([[x[0],xAntLM[0]],[x[1],xAntLM[1]]])  
    #Establecemos como condicion de paro, que la diferencia fx_anterior-fx_actual sea menor a epsilon o que la norma del gradiente sea menor a epsilon
    if(abs(fx_prev - fx) < ep1 or LA.norm(G)<ep1):
        print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x[0], x[1],fx_prev,LA.norm(G)))
        break
    print('{0}\t[{1:.4f},{2:.4f}]\t\t{3:.4f}\t\t{4:.4f}'.format(i, x[0], x[1],fx_prev,LA.norm(G)))
    fx_prev=fx #Ahora fx sera fx anterior     
    iteraciones+=1
elapsed_time = (time() - start_time)*1000
print("Tiempo transcurrido: %f" % elapsed_time)
print("Iteraciones: %d" % iteraciones)
#Graficamos el resultado del Steepest descent method
graficaUnaaUna(puntosM, 'Marquardt’s method') 

Valor de la función inicial = 270.294169 
0	[-0.9778,0.9634]		270.2942		444.3158
1	[-0.9363,0.8955]		251.6203		428.0979
2	[-0.8635,0.7779]		218.7471		397.9601
3	[-0.7492,0.5990]		167.3425		345.6663
4	[-0.5978,0.3806]		102.4403		265.5825
5	[-0.4329,0.1956]		45.0861		166.5017
6	[-0.2585,0.1152]		14.8448		79.3220
7	[-0.0282,0.1225]		4.5436		35.1505
8	[0.2325,0.1119]		-1.4682		22.1993
9	[0.3997,0.0554]		-5.6152		14.1528
10	[0.4757,0.0147]		-7.0701		6.2961
11	[0.4968,0.0015]		-7.3353		1.6001
12	[0.4997,-0.0002]		-7.3527		0.1763
Tiempo transcurrido: 3.970623
Iteraciones: 12
